# 03 BMRA

Run BMRA based on global responses derived from the pathway activity predictions.

In [26]:
import pandas as pd
import numpy as np
import os
import bmra

In [27]:
data_dir_1 = "01_outputs_2020"
data_dir_2 = "02_outputs_2020"
info_dir = "~/Phd_project/project_GBM/gbm_DATA/gbm_DATA_LINCS_GI1"
out_dir = "03_outputs_2020"


os.makedirs(out_dir, exist_ok = True)

## Load data

### Global responses

Assembled from global responses calculated from predicted pathway activities, and DPD global responses.

In [28]:
def assemble_R_global(pathway_rglobal_file, dpd_rglobal_file):
    """
    Assemble full R_global matrix
    """
    R_global_modules_df = pd.read_csv(pathway_rglobal_file, index_col = 0)

    perts = R_global_modules_df.columns.tolist()

    R_global_DPD_df = pd.read_csv(dpd_rglobal_file, index_col=0).T

    # filter by perts list
    R_global_DPD_df = R_global_DPD_df[R_global_DPD_df.index.isin(perts)]

    # sort by perts list
    R_global_DPD_df["sort_col"] = R_global_DPD_df.index.map({val: i for i, val in enumerate(perts)})
    R_global_DPD_df = R_global_DPD_df.sort_values("sort_col")
    R_global_DPD_df = R_global_DPD_df.drop("sort_col", axis = 1)

    # transpose
    R_global_DPD_df = R_global_DPD_df.T

    R_global_df = pd.concat([R_global_modules_df, R_global_DPD_df])

    return R_global_df

In [29]:
R_global_df = assemble_R_global(
    pathway_rglobal_file = os.path.join(data_dir_2, "R_global_annotated.csv"),
    dpd_rglobal_file = os.path.join(data_dir_1, "R_global_DPDonly_annotated.csv"),
)

modules = R_global_df.index.tolist()
exp_ids = R_global_df.columns.tolist()

R_global = R_global_df.values
n_modules = R_global.shape[0]

print(R_global.shape)
display(R_global_df)

(13, 119)


,ASG002_GI1_24H:A10,ASG002_GI1_24H:A11,ASG002_GI1_24H:A12,ASG002_GI1_24H:A19,ASG002_GI1_24H:A20,ASG002_GI1_24H:A21,ASG002_GI1_24H:B10,ASG002_GI1_24H:B11,ASG002_GI1_24H:B12,ASG002_GI1_24H:B13,...,MOAR010_GI1_24H:J05,MOAR010_GI1_24H:J06,MOAR010_GI1_24H:J22,MOAR010_GI1_24H:J23,MOAR010_GI1_24H:J24,MOAR010_GI1_24H:K08,MOAR010_GI1_24H:K09,MOAR011_GI1_24H:F07,MOAR011_GI1_24H:F08,MOAR011_GI1_24H:F09
CDK1,-0.532456,-0.090838,-0.000091,0.002085,-0.165900,-0.121836,-0.068890,-0.324622,-0.246149,-0.447347,...,0.010588,0.087241,-0.291894,-0.010809,-0.191546,-0.152258,-0.030956,-0.325711,-0.075607,0.065381
CDK2,-0.005915,0.058437,-0.005472,-0.468645,-0.258947,-0.051079,0.083799,-0.149857,0.069219,0.036831,...,-0.089429,0.014765,-0.007504,-0.235402,0.066590,0.036492,-0.111734,0.021896,0.011417,0.041343
CDK4_6,-1.020350,-0.156489,0.006785,-0.309805,-0.362326,-0.206513,-0.273633,-0.376785,-0.431937,-1.081409,...,0.123113,-0.029467,-0.008531,0.122665,-0.161915,-0.206933,-0.269666,-0.194807,-0.201761,0.131028
PI3K,-1.980469,-1.511715,-1.282948,-1.765735,-1.240682,-0.539564,-0.150972,-0.257132,-0.615737,-0.564920,...,0.144486,-0.325722,-0.181422,-0.156620,-0.412223,-0.142093,-0.242985,-0.738465,-0.335861,0.070843
ERK,-1.069782,-0.434094,-0.327457,-0.627189,-0.324659,-0.664645,-0.421138,-0.976800,-0.589117,-1.109856,...,-0.065764,0.098499,-0.047844,0.108805,0.041457,-0.089113,-0.085854,-0.305833,-0.370252,-0.017460
Hypoxia,-0.254387,0.023142,0.058141,0.041423,-0.035411,-0.038288,0.084564,-0.109133,-0.036479,-0.157980,...,-0.116061,-0.009096,0.055604,-0.090556,-0.062510,-0.037085,0.081561,-0.022819,-0.004053,-0.022677
PDGFR,-1.047116,0.085648,0.249960,-0.016258,-0.287235,-0.056367,0.180483,-0.500696,0.003939,-0.280822,...,0.174566,-0.001951,0.210924,0.135909,0.031303,0.232381,0.105209,-0.087007,-0.096901,0.006698
Aurora,-0.176946,0.165538,0.098505,0.043972,-0.156947,-0.192212,-0.368580,-0.472288,0.026675,-0.881248,...,0.098747,0.135528,0.186694,0.251682,0.001502,0.193990,-0.003241,-0.354541,-0.150168,0.353166
Estrogen,-0.059273,-0.011432,-0.115573,0.024551,0.167143,-0.594219,-0.040831,0.054088,-0.049902,0.112370,...,0.133559,0.177735,0.177713,0.137463,-0.492810,0.212787,-0.405344,-0.401177,0.000818,-0.041655
EGFR,-0.397070,-0.016259,0.172429,-0.192615,-0.109157,0.116712,0.128445,-0.615233,0.143768,-1.271298,...,0.054318,-0.351044,-0.453752,-0.124671,-0.112722,-0.202544,-0.215714,-0.202577,-0.156586,0.036562


In [30]:
print(R_global_df.values.min())
print(R_global_df.values.max())

-13.504171722962724
7.677933039891711


### Perturbation matrix

In [31]:
inhib_conc_df = pd.read_csv(os.path.join(data_dir_1, "inhib_conc_annotated.csv"), index_col = 0)
#LFC_PLCg = pd.read_csv(os.path.join(info_dir,"PLCg_Data_log_2020.csv"),index_col=0)
#inhib_conc = inhib_conc_df.drop(columns=LFC_PLCg.index).values
inhib_conc = inhib_conc_df.values

n_DPDs = R_global.shape[0] - inhib_conc.shape[0]

pert_modules = np.where(inhib_conc != 0, 1, 0)
pert_DPD = np.zeros((n_DPDs, len(exp_ids)))

pert = np.vstack([pert_modules, pert_DPD])

print(pert.shape)
print(pert)

(13, 119)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Networks

In [32]:
# empty prior
#G_prior = np.zeros((n_modules, n_modules))
G_prior_df = pd.read_excel(os.path.join(info_dir,"ALL_DATA_2020_Jing_gbm.xlsx"), sheet_name = "prior_network", index_col = 0)

G_prior = G_prior_df.values
G_prior_df

,CDK1,CDK2,CDK4_6,p53,EGFR,Aurora,Estrogen,PDGFR,Hypoxia,ERK,PI3K,GBM_onc,GBM_survival
CDK1,0,0,0,0,0,0,0,0,0,0,0,0,0
CDK2,0,0,0,0,0,0,0,0,0,0,0,0,0
CDK4_6,0,0,0,0,0,0,0,0,0,0,0,0,0
p53,0,0,0,0,0,0,0,0,0,0,0,0,0
EGFR,0,0,0,0,0,0,0,0,0,0,0,0,0
Aurora,0,0,0,0,0,0,0,0,0,0,0,0,0
Estrogen,0,0,0,0,0,0,0,0,0,0,0,0,0
PDGFR,0,0,0,0,0,0,0,0,0,0,0,0,0
Hypoxia,0,0,0,0,0,0,0,0,0,0,0,0,0
ERK,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
#G_prior_r_df = pd.read_excel(os.path.join(info_dir, "ALL_DATA_2020.xlsx"), sheet_name = "prior_r", index_col = 0)

#G_prior_r = G_prior_r_df.values
#G_prior_r_df

In [34]:
G_not = np.eye(n_modules)
G_not[:, n_modules - n_DPDs :] = 1

G_not

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]])

## Run BMRA

In [35]:
n_gibbs = 200_000
n_window = 5_000

mode = "all"
n_iterations = 1

In [36]:
G_prior

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [37]:
A_mean, A_sd, r_mean, r_sd = bmra.run_bmra(R_global,
            pert,
            G_prior,
            G_not,
            #r_prior = G_prior_r,
            n_gibbs = n_gibbs,
            n_window = n_window,
            mode = mode,
            n_iterations = n_iterations,
        )

A_mean_df = pd.DataFrame(A_mean, index = modules, columns = modules)
A_sd_df = pd.DataFrame(A_sd, index = modules, columns = modules)
r_mean_df = pd.DataFrame(r_mean, index = modules, columns = modules)
r_sd_df = pd.DataFrame(r_sd, index = modules, columns = modules)

display(A_mean_df)
display(A_sd_df)
display(r_mean_df)
display(r_sd_df)

A_mean_df.to_csv(os.path.join(out_dir, "A_mean.csv"))
A_sd_df.to_csv(os.path.join(out_dir, "A_sd.csv"))
r_mean_df.to_csv(os.path.join(out_dir, "r_mean.csv"))
r_sd_df.to_csv(os.path.join(out_dir, "r_sd.csv"))

,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,0.000000,1.000000,1.000000,1.000000,1.000000,0.000318,0.001130,0.002290,0.001026,0.002757,0.025501,0.0,0.0
CDK2,1.000000,0.000000,0.000000,0.000000,0.999370,0.117025,0.001245,0.000000,0.000000,0.000000,0.000630,0.0,0.0
CDK4_6,1.000000,0.000000,0.000000,1.000000,0.894080,0.000000,1.000000,0.184332,0.000000,0.182057,0.000000,0.0,0.0
PI3K,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ERK,1.000000,0.000000,0.177925,0.779603,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.0
Hypoxia,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
PDGFR,0.000000,1.000000,1.000000,0.262851,0.799464,0.122742,0.000000,0.000412,0.000000,1.000000,1.000000,0.0,0.0
Aurora,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.005848,0.0,0.0
Estrogen,0.490868,0.000162,0.046067,0.004350,0.022027,0.000503,0.791297,0.253213,0.000000,0.000547,0.269470,0.0,0.0
EGFR,0.949308,0.000000,0.000000,0.000000,0.000000,0.003580,1.000000,1.000000,0.099112,0.000000,0.000000,0.0,0.0


,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,0.000000e+00,2.442491e-15,2.442491e-15,2.442491e-15,2.442491e-15,1.784141e-02,3.359888e-02,4.779738e-02,3.202202e-02,5.243699e-02,1.576415e-01,0.0,0.0
CDK2,2.220446e-16,0.000000e+00,0.000000e+00,0.000000e+00,2.508856e-02,3.214499e-01,3.526097e-02,0.000000e+00,0.000000e+00,0.000000e+00,2.508856e-02,0.0,0.0
CDK4_6,8.881784e-16,0.000000e+00,0.000000e+00,8.881784e-16,3.077356e-01,0.000000e+00,8.881784e-16,3.877544e-01,0.000000e+00,3.858917e-01,0.000000e+00,0.0,0.0
PI3K,0.000000e+00,2.997602e-15,2.997602e-15,0.000000e+00,2.997602e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
ERK,3.441691e-15,0.000000e+00,3.824492e-01,4.145142e-01,0.000000e+00,0.000000e+00,3.441691e-15,3.441691e-15,0.000000e+00,0.000000e+00,3.441691e-15,0.0,0.0
Hypoxia,0.000000e+00,0.000000e+00,0.000000e+00,3.330669e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
PDGFR,0.000000e+00,2.220446e-15,2.220446e-15,4.401823e-01,4.004011e-01,3.281403e-01,0.000000e+00,2.030422e-02,0.000000e+00,2.220446e-15,2.220446e-15,0.0,0.0
Aurora,0.000000e+00,0.000000e+00,3.552714e-15,0.000000e+00,3.552714e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.552714e-15,7.625132e-02,0.0,0.0
Estrogen,4.999166e-01,1.273728e-02,2.096305e-01,6.581015e-02,1.467719e-01,2.242471e-02,4.063820e-01,4.348520e-01,0.000000e+00,2.338229e-02,4.436843e-01,0.0,0.0
EGFR,2.193682e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.972226e-02,1.776357e-15,1.776357e-15,2.988117e-01,0.000000e+00,0.000000e+00,0.0,0.0


,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,0.000000,0.328181,0.238801,0.118758,0.098388,0.000005,0.000011,-0.000051,-0.000052,-0.000043,-0.000924,0.0,0.0
CDK2,0.427041,0.000000,0.000000,0.000000,-0.099878,0.009966,-0.000088,0.000000,0.000000,0.000000,-0.000036,0.0,0.0
CDK4_6,0.362319,0.000000,0.000000,0.144459,0.127057,0.000000,0.204281,0.020432,0.000000,0.016459,0.000000,0.0,0.0
PI3K,0.000000,0.536471,0.292597,0.000000,0.184380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ERK,0.453512,0.000000,0.043403,0.129407,0.000000,0.000000,0.257416,0.288739,0.000000,0.000000,0.177919,0.0,0.0
Hypoxia,0.000000,0.000000,0.000000,0.176581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
PDGFR,0.000000,-0.414302,0.400113,0.051051,0.143038,0.019988,0.000000,0.000033,0.000000,0.248909,-0.229390,0.0,0.0
Aurora,0.000000,0.000000,0.288571,0.000000,0.235337,0.000000,0.000000,0.000000,0.000000,0.263507,-0.000556,0.0,0.0
Estrogen,-0.078994,-0.000013,-0.006370,-0.000356,-0.002238,-0.000025,0.084275,0.031130,0.000000,0.000003,0.024521,0.0,0.0
EGFR,0.231879,0.000000,0.000000,0.000000,0.000000,0.000439,0.177920,0.176463,-0.021962,0.000000,0.000000,0.0,0.0


,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,0.000000,0.074171,0.038015,0.031086,0.026469,0.000348,0.000502,0.001363,0.001805,0.001345,0.006579,0.0,0.0
CDK2,0.017818,0.000000,0.000000,0.000000,0.009539,0.028261,0.002648,0.000000,0.000000,0.000000,0.001426,0.0,0.0
CDK4_6,0.047698,0.000000,0.000000,0.031523,0.051194,0.000000,0.028209,0.044886,0.000000,0.036890,0.000000,0.0,0.0
PI3K,0.000000,0.025085,0.019139,0.000000,0.013141,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ERK,0.057392,0.000000,0.095119,0.076943,0.000000,0.000000,0.033398,0.033617,0.000000,0.000000,0.027543,0.0,0.0
Hypoxia,0.000000,0.000000,0.000000,0.001365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
PDGFR,0.000000,0.090783,0.060204,0.088805,0.080504,0.055785,0.000000,0.001670,0.000000,0.035677,0.032148,0.0,0.0
Aurora,0.000000,0.000000,0.034334,0.000000,0.023965,0.000000,0.000000,0.000000,0.000000,0.022282,0.007302,0.0,0.0
Estrogen,0.086237,0.000983,0.030443,0.005786,0.015718,0.001314,0.053071,0.056209,0.000000,0.000372,0.043053,0.0,0.0
EGFR,0.075303,0.000000,0.000000,0.000000,0.000000,0.007494,0.039433,0.043923,0.068173,0.000000,0.000000,0.0,0.0


In [38]:
np.fill_diagonal(r_mean_df.values,-1)
rm_minus_inv = pd.DataFrame(np.linalg.pinv(r_mean_df),index=r_mean_df.index,columns=r_mean_df.columns)*(-1)
rm_minus_inv.to_csv(os.path.join(out_dir, 'r_minv.csv'))
display(rm_minus_inv)

,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,1.661085,0.635356,0.633467,0.341479,0.303319,0.010869,0.224408,0.117535,-0.001906,0.097183,0.000819,1.026550e-15,7.577322e-16
CDK2,0.582404,1.227654,0.179800,0.095204,-0.022187,0.012944,0.035129,-0.000904,-0.005183,0.011439,-0.012715,6.962050e-16,2.953028e-17
CDK4_6,1.017830,0.365651,1.653682,0.445721,0.470557,0.013748,0.500838,0.205432,-0.017166,0.205969,-0.032655,3.954886e-16,8.769376e-16
PI3K,0.847267,0.848993,0.750081,1.279038,0.407094,0.014079,0.278132,0.154411,0.000253,0.122227,0.007736,5.440029e-16,5.288446e-16
ERK,1.285445,0.452346,0.920716,0.529053,1.525724,0.016880,0.611469,0.514089,0.043697,0.302766,0.130771,5.330482e-16,8.359695e-16
Hypoxia,0.149611,0.149916,0.132450,0.225853,0.071885,1.002486,0.049113,0.027266,0.000045,0.021583,0.001366,3.210848e-16,1.761730e-16
PDGFR,0.512654,-0.255636,0.856677,0.297576,0.452544,0.025774,1.408445,0.219508,-0.104027,0.422491,-0.245704,-7.079491e-16,3.032292e-16
Aurora,0.757311,0.250749,0.810203,0.301890,0.560374,0.010404,0.385566,1.255821,-0.005967,0.440191,0.009702,-4.088587e-16,9.552374e-16
Estrogen,-0.070294,-0.063197,0.035909,0.006539,0.030132,0.001467,0.105294,0.047556,1.001377,0.039337,0.005803,-1.024579e-16,-2.449143e-16
EGFR,0.611629,0.147545,0.441548,0.185355,0.249105,0.009349,0.368374,0.286883,-0.041995,1.174527,-0.041941,-1.611821e-16,4.274690e-16


In [39]:
con_mat = pd.DataFrame(columns=['From','To','Strength'])
for con_to in r_mean_df.index:
    for con_from in r_mean_df.columns:
        if ((r_mean_df.loc[con_to,con_from] != 0) & (con_from != con_to)):
            row_df = pd.DataFrame([[con_from,con_to,r_mean_df.loc[con_to,con_from]]],columns=['From','To','Strength'])
            if con_mat.empty:
                con_mat = row_df.copy()
            else:
                con_mat = pd.concat([con_mat,row_df],axis=0,ignore_index=True)
            #con_mat = con_mat.append({'From':con_from,'To':con_to,'Strength':r_mean_df.loc[con_to,con_from]},ignore_index=True)
con_mat.to_csv(os.path.join(out_dir, 'r_net.txt'),sep='\t',index=False)
display(con_mat)

,From,To,Strength
0,CDK2,CDK1,0.328181
1,CDK4_6,CDK1,0.238801
2,PI3K,CDK1,0.118758
3,ERK,CDK1,0.098388
4,Hypoxia,CDK1,0.000005
...,...,...,...
69,Aurora,GBM_onc,-0.238533
70,EGFR,GBM_onc,-0.000023
71,p53,GBM_onc,0.013803
72,ERK,GBM_survival,1.507120


In [40]:
# now the same but with filtered matrices
# threshold value
Athr = 0.5

rm_filt = r_mean_df[np.abs(A_mean_df) > Athr].fillna(0).copy()
np.fill_diagonal(rm_filt.values,-1)
rm_filt.to_csv(os.path.join(out_dir, 'r_mean_filt.csv'))
display(rm_filt)

rm_filt_minus_inv = pd.DataFrame(np.linalg.pinv(rm_filt),index=rm_filt.index,columns=rm_filt.columns)*(-1)
rm_filt_minus_inv.to_csv(os.path.join(out_dir, 'r_minv_filt.csv'))
display(rm_minus_inv)

con_mat = pd.DataFrame(columns=['From','To','Strength'])
for con_to in rm_filt.index:
    for con_from in rm_filt.columns:
        if ((rm_filt.loc[con_to,con_from] != 0) & (con_from != con_to)):
            row_df = pd.DataFrame([[con_from,con_to,rm_filt.loc[con_to,con_from]]],columns=['From','To','Strength'])
            if con_mat.empty:
                con_mat = row_df.copy()
            else:
                con_mat = pd.concat([con_mat,row_df],axis=0,ignore_index=True)
            #con_mat = con_mat.append({'From':con_from,'To':con_to,'Strength':r_mean_df.loc[con_to,con_from]},ignore_index=True)
con_mat.to_csv(os.path.join(out_dir, 'r_net_filt.txt'),sep='\t',index=False)
#display(con_mat)


,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,-1.000000,0.328181,0.238801,0.118758,0.098388,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0
CDK2,0.427041,-1.000000,0.000000,0.000000,-0.099878,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0
CDK4_6,0.362319,0.000000,-1.000000,0.144459,0.127057,0.000000,0.204281,0.000000,0.00000,0.000000,0.000000,0.0,0.0
PI3K,0.000000,0.536471,0.292597,-1.000000,0.184380,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0
ERK,0.453512,0.000000,0.000000,0.129407,-1.000000,0.000000,0.257416,0.288739,0.00000,0.000000,0.177919,0.0,0.0
Hypoxia,0.000000,0.000000,0.000000,0.176581,0.000000,-1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0
PDGFR,0.000000,-0.414302,0.400113,0.000000,0.143038,0.000000,-1.000000,0.000000,0.00000,0.248909,-0.229390,0.0,0.0
Aurora,0.000000,0.000000,0.288571,0.000000,0.235337,0.000000,0.000000,-1.000000,0.00000,0.263507,0.000000,0.0,0.0
Estrogen,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084275,0.000000,-1.00000,0.000000,0.000000,0.0,0.0
EGFR,0.231879,0.000000,0.000000,0.000000,0.000000,0.000000,0.177920,0.176463,0.00000,-1.000000,0.000000,0.0,0.0


,CDK1,CDK2,CDK4_6,PI3K,ERK,Hypoxia,PDGFR,Aurora,Estrogen,EGFR,p53,GBM_onc,GBM_survival
CDK1,1.661085,0.635356,0.633467,0.341479,0.303319,0.010869,0.224408,0.117535,-0.001906,0.097183,0.000819,1.026550e-15,7.577322e-16
CDK2,0.582404,1.227654,0.179800,0.095204,-0.022187,0.012944,0.035129,-0.000904,-0.005183,0.011439,-0.012715,6.962050e-16,2.953028e-17
CDK4_6,1.017830,0.365651,1.653682,0.445721,0.470557,0.013748,0.500838,0.205432,-0.017166,0.205969,-0.032655,3.954886e-16,8.769376e-16
PI3K,0.847267,0.848993,0.750081,1.279038,0.407094,0.014079,0.278132,0.154411,0.000253,0.122227,0.007736,5.440029e-16,5.288446e-16
ERK,1.285445,0.452346,0.920716,0.529053,1.525724,0.016880,0.611469,0.514089,0.043697,0.302766,0.130771,5.330482e-16,8.359695e-16
Hypoxia,0.149611,0.149916,0.132450,0.225853,0.071885,1.002486,0.049113,0.027266,0.000045,0.021583,0.001366,3.210848e-16,1.761730e-16
PDGFR,0.512654,-0.255636,0.856677,0.297576,0.452544,0.025774,1.408445,0.219508,-0.104027,0.422491,-0.245704,-7.079491e-16,3.032292e-16
Aurora,0.757311,0.250749,0.810203,0.301890,0.560374,0.010404,0.385566,1.255821,-0.005967,0.440191,0.009702,-4.088587e-16,9.552374e-16
Estrogen,-0.070294,-0.063197,0.035909,0.006539,0.030132,0.001467,0.105294,0.047556,1.001377,0.039337,0.005803,-1.024579e-16,-2.449143e-16
EGFR,0.611629,0.147545,0.441548,0.185355,0.249105,0.009349,0.368374,0.286883,-0.041995,1.174527,-0.041941,-1.611821e-16,4.274690e-16


In [41]:
# number of zero elements in the matrix
display(len(rm_filt.index)*len(rm_filt.columns) - rm_filt.astype(bool).sum().sum())

116

In [42]:
# number of non-zero elements in the matrix
display(rm_filt.astype(bool).sum().sum() - len(modules))

40